# Mini-Challenge 2 npr
## Q&A

In [1]:
import json
import numpy as np
import pandas as pd
from datasets import load_dataset, load_metric
import torch
from transformers import AutoTokenizer, AutoModelForQuestionAnswering, TrainingArguments, Trainer, default_data_collator
from utils import *

config = load_yaml_file('electra_config.yaml')

##
GPU_USAGE = config['GPU_USAGE']
BATCH_SIZE = config['BATCH_SIZE']
MAX_LENGTH = config['MAX_LENGTH']
DOC_STRIDE = config['DOC_STRIDE']
##

tokenizer = AutoTokenizer.from_pretrained("google/electra-small-discriminator")
model = AutoModelForQuestionAnswering.from_pretrained("google/electra-small-discriminator").to("cuda:0")

PAD_RIGHT = tokenizer.padding_side == "right"

Some weights of the model checkpoint at xlnet-base-cased were not used when initializing XLNetForQuestionAnsweringSimple: ['lm_loss.weight', 'lm_loss.bias']
- This IS expected if you are initializing XLNetForQuestionAnsweringSimple from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLNetForQuestionAnsweringSimple from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLNetForQuestionAnsweringSimple were not initialized from the model checkpoint at xlnet-base-cased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [2]:
torch.cuda.is_available()

True

# Load Data

In [3]:
data = load_dataset("squad_v2")
# with open('data/train-v2.0.json') as f:
#     data = json.load(f)

Reusing dataset squad_v2 (C:\Users\vince\.cache\huggingface\datasets\squad_v2\squad_v2\2.0.0\09187c73c1b837c95d9a249cd97c2c3f1cebada06efe667b4427714b27639b1d)
100%|██████████| 2/2 [00:00<00:00, 82.85it/s]


In [4]:
tokenized_datasets = data.map(
    prepare_train_features, 
    fn_kwargs={
        'tokenizer':tokenizer, 
        'PAD_RIGHT':PAD_RIGHT,
        'MAX_LENGTH':MAX_LENGTH, 
        'DOC_STRIDE':DOC_STRIDE
        }, 
    batched=True,
    remove_columns=data["train"].column_names)

Loading cached processed dataset at C:\Users\vince\.cache\huggingface\datasets\squad_v2\squad_v2\2.0.0\09187c73c1b837c95d9a249cd97c2c3f1cebada06efe667b4427714b27639b1d\cache-f7f821988ca865d4.arrow
Loading cached processed dataset at C:\Users\vince\.cache\huggingface\datasets\squad_v2\squad_v2\2.0.0\09187c73c1b837c95d9a249cd97c2c3f1cebada06efe667b4427714b27639b1d\cache-c9925728a42415f7.arrow


# Train Model

In [7]:
model_name = "google/electra-small-discriminator".split("/")[-1]

import os
os.environ["WANDB_DISABLED"] = "true"

args = TrainingArguments(
    f"{model_name}-finetuned-squad",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    num_train_epochs=3,
    weight_decay=0.01,
    push_to_hub=False,

)

data_collator = default_data_collator

trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
)

trainer.train()

Using the `WAND_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
***** Running training *****
  Num examples = 137985
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 51747
  1%|          | 500/51747 [02:55<5:02:51,  2.82it/s]Saving model checkpoint to xlnet-base-cased-finetuned-squad\checkpoint-500
Configuration saved in xlnet-base-cased-finetuned-squad\checkpoint-500\config.json


{'loss': 1.3331, 'learning_rate': 1.9806752082246313e-05, 'epoch': 0.03}


Model weights saved in xlnet-base-cased-finetuned-squad\checkpoint-500\pytorch_model.bin
tokenizer config file saved in xlnet-base-cased-finetuned-squad\checkpoint-500\tokenizer_config.json
Special tokens file saved in xlnet-base-cased-finetuned-squad\checkpoint-500\special_tokens_map.json
  2%|▏         | 1000/51747 [05:56<4:55:55,  2.86it/s]Saving model checkpoint to xlnet-base-cased-finetuned-squad\checkpoint-1000
Configuration saved in xlnet-base-cased-finetuned-squad\checkpoint-1000\config.json


{'loss': 0.8253, 'learning_rate': 1.9613504164492628e-05, 'epoch': 0.06}


Model weights saved in xlnet-base-cased-finetuned-squad\checkpoint-1000\pytorch_model.bin
tokenizer config file saved in xlnet-base-cased-finetuned-squad\checkpoint-1000\tokenizer_config.json
Special tokens file saved in xlnet-base-cased-finetuned-squad\checkpoint-1000\special_tokens_map.json
  3%|▎         | 1500/51747 [08:52<4:51:30,  2.87it/s] Saving model checkpoint to xlnet-base-cased-finetuned-squad\checkpoint-1500
Configuration saved in xlnet-base-cased-finetuned-squad\checkpoint-1500\config.json


...

Model weights saved in xlnet-base-cased-finetuned-squad\checkpoint-51000\pytorch_model.bin
tokenizer config file saved in xlnet-base-cased-finetuned-squad\checkpoint-51000\tokenizer_config.json
Special tokens file saved in xlnet-base-cased-finetuned-squad\checkpoint-51000\special_tokens_map.json
100%|█████████▉| 51500/51747 [4:54:13<01:21,  3.04it/s]Saving model checkpoint to xlnet-base-cased-finetuned-squad\checkpoint-51500
Configuration saved in xlnet-base-cased-finetuned-squad\checkpoint-51500\config.json


{'loss': 0.2081, 'learning_rate': 9.546447137032099e-08, 'epoch': 2.99}


Model weights saved in xlnet-base-cased-finetuned-squad\checkpoint-51500\pytorch_model.bin
tokenizer config file saved in xlnet-base-cased-finetuned-squad\checkpoint-51500\tokenizer_config.json
Special tokens file saved in xlnet-base-cased-finetuned-squad\checkpoint-51500\special_tokens_map.json
100%|█████████▉| 51746/51747 [4:55:36<00:00,  3.05it/s]***** Running Evaluation *****
  Num examples = 12852
  Batch size = 8

100%|██████████| 51747/51747 [4:59:13<00:00,  3.05it/s]

Training completed. Do not forget to share your model on huggingface.co/models =)


100%|██████████| 51747/51747 [4:59:13<00:00,  2.88it/s]

{'eval_loss': 0.5763306617736816, 'eval_runtime': 217.0592, 'eval_samples_per_second': 59.21, 'eval_steps_per_second': 7.404, 'epoch': 3.0}
{'train_runtime': 17953.3942, 'train_samples_per_second': 23.057, 'train_steps_per_second': 2.882, 'train_loss': 0.3545851099190948, 'epoch': 3.0}


TrainOutput(global_step=51747, training_loss=0.3545851099190948, metrics={'train_runtime': 17953.3942, 'train_samples_per_second': 23.057, 'train_steps_per_second': 2.882, 'train_loss': 0.3545851099190948, 'epoch': 3.0})

In [8]:
trainer.save_model("test-squad-trained_electra")

Saving model checkpoint to test-squad-trained_xlnet_RETRAINED
Configuration saved in test-squad-trained_xlnet_RETRAINED\config.json
Model weights saved in test-squad-trained_xlnet_RETRAINED\pytorch_model.bin
tokenizer config file saved in test-squad-trained_xlnet_RETRAINED\tokenizer_config.json
Special tokens file saved in test-squad-trained_xlnet_RETRAINED\special_tokens_map.json
